In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [4]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [5]:
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import miceforest as mf
%xmode Context

Exception reporting mode: Context


In [6]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(6)
pl.enable_string_cache()

In [7]:
path_to_train = "/home/sohail/Downloads/credit_risk/train/"
path_to_test = "/home/sohail/Downloads/credit_risk/test/"
path_to_features = "/home/sohail/Downloads/feature_definitions.csv"

In [8]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [9]:
all_train_files = glob(path_to_train+"/*.parquet")
all_test_files = glob(path_to_test+"/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""/home/sohail/Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""/home/sohail/Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""/home/sohail/Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""/home/sohail/Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""/home/sohail/Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""/home/sohail/Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""/home/sohail/Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [10]:
display(train_files_df.filter(pl.col("filename").str.contains("base")))
_ind = train_files_df.filter(pl.col("filename").str.contains("base")).select("index").item()

index,path,filename
i64,str,str
7,"""/home/sohail/Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [11]:
train_base = (
    pl.read_parquet(all_train_files[_ind])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
train_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
0,2019-01-03,1,1,4,0,0
1,2019-01-03,1,1,4,0,0
2,2019-01-04,1,1,5,0,0
…,…,…,…,…,…,…
"2,703,452",2020-10-05,10,41,1,1,0
"2,703,453",2020-10-05,10,41,1,1,0
"2,703,454",2020-10-05,10,41,1,1,0


In [12]:
display(test_files_df.filter(pl.col("filename").str.contains("base")))
_ind = test_files_df.filter(pl.col("filename").str.contains("base")).select("index").item()

index,path,filename
i64,str,str
33,"""/home/sohail/Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [13]:
test_base = (
    pl.read_parquet(all_test_files[_ind])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("Date"),
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.week().alias("week"),
        pl.col("Date").dt.weekday().alias("weekday"),
        (pl.col("Date").dt.year() - 2019).alias("year"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
)
test_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
"57,543",2020-10-06,10,41,2,1,0
"57,549",2020-10-06,10,41,2,1,0
"57,551",2020-10-06,10,41,2,1,0
…,…,…,…,…,…,…
"57,632",2020-10-06,10,41,2,1,0
"57,633",2020-10-06,10,41,2,1,0
"57,634",2020-10-06,10,41,2,1,0


In [14]:
all_files_df = (
    pl.concat(
        [
            train_files_df,
            test_files_df
        ]
    )
    .sort(by="filename")
)
all_files_df.head()

index,path,filename
i64,str,str
31,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""/home/sohail/Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""/home/sohail/Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [15]:
total_base = (
    pl.concat(
        [
            train_base,
            test_base
        ]
    )
)
total_base

case_id_base,Date,month,week,weekday,year,target
u32,date,i8,i8,i8,i32,u8
0,2019-01-03,1,1,4,0,0
1,2019-01-03,1,1,4,0,0
2,2019-01-04,1,1,5,0,0
…,…,…,…,…,…,…
"57,632",2020-10-06,10,41,2,1,0
"57,633",2020-10-06,10,41,2,1,0
"57,634",2020-10-06,10,41,2,1,0


In [16]:
def reduce_dtypes(df:pl.DataFrame):
    return(
        df
    )
def grouping(df:pl.DataFrame):
    return (
        df
        .group_by("case_id")
        .agg(
            cs.numeric().max(),
            (~cs.numeric()).drop_nulls().mode().first()
        )
    )

In [17]:
def read_and_cast(filter_string:str,prefix_string:str):
    train_files_list = sorted(train_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list())
    test_files_list = sorted(test_files_df.filter(pl.col("filename").str.contains(filter_string))["path"].to_list())
    train_df = (
        pl.concat(
            [
                pl.read_parquet(_).pipe(reduce_dtypes).pipe(grouping) for _ in train_files_list
            ]
        )
    )
    test_df = (
        pl.concat(
            [
                pl.read_parquet(_).select(train_df.columns).cast(train_df.schema).pipe(reduce_dtypes).pipe(grouping) for _ in test_files_list
            ]
        )
    )
    return (
        pl.concat(
            [train_df,test_df],
            how="vertical_relaxed"
        )
        .rename({_:f"{prefix_string}_{_}" for _ in train_df.columns if not _ == "case_id"})
    )

def select_impuatable(df:pl.DataFrame,thresh=0.95):
    cols =  (
        df
        .select(pl.all().is_null().mean())
        .transpose(include_header=True)
        .filter(pl.col("column_0") < thresh)
        ["column"].to_list()
    )
    return df.select(cols)

## Past Application

In [20]:
total_past_shallow = (
    read_and_cast("applprev_1_","past_shallow")
)
total_past_shallow

case_id,past_shallow_actualdpd_943P,past_shallow_annuity_853A,past_shallow_byoccupationinc_3656910L,past_shallow_childnum_21L,past_shallow_credacc_actualbalance_314A,past_shallow_credacc_credlmt_575A,past_shallow_credacc_maxhisbal_375A,past_shallow_credacc_minhisbal_90A,past_shallow_credacc_transactions_402L,past_shallow_credamount_590A,past_shallow_currdebt_94A,past_shallow_downpmt_134A,past_shallow_mainoccupationinc_437A,past_shallow_maxdpdtolerance_577P,past_shallow_num_group1,past_shallow_outstandingdebt_522A,past_shallow_pmtnum_8L,past_shallow_revolvingaccount_394A,past_shallow_tenor_203L,past_shallow_approvaldate_319D,past_shallow_cancelreason_3545846M,past_shallow_creationdate_885D,past_shallow_credacc_status_367L,past_shallow_credtype_587L,past_shallow_dateactivated_425D,past_shallow_district_544M,past_shallow_dtlastpmt_581D,past_shallow_dtlastpmtallstes_3545839D,past_shallow_education_1138M,past_shallow_employedfrom_700D,past_shallow_familystate_726L,past_shallow_firstnonzeroinstldate_307D,past_shallow_inittransactioncode_279L,past_shallow_isbidproduct_390L,past_shallow_isdebitcard_527L,past_shallow_postype_4733339M,past_shallow_profession_152M,past_shallow_rejectreason_755M,past_shallow_rejectreasonclient_4145042M,past_shallow_status_219L
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u16,f32,f32,f32,f32,date,cat,date,str,str,date,cat,date,date,cat,date,str,date,str,bool,bool,cat,cat,cat,cat,str
"1,256,284",0.00,"6,574.00","25,000.00",1.00,null,0.00,null,null,null,"48,998.00",0.00,0.00,"80,000.00",26.00,14,0.00,18.00,null,18.00,2014-09-05,"""a55475b1""",2012-08-16,null,"""COL""",2014-02-11,"""P142_57_166""",2018-12-26,2018-06-09,"""a55475b1""",2010-07-15,"""MARRIED""",2017-03-29,"""POS""",false,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
"1,479,000",0.00,"2,000.60",1.00,0.00,"36,241.80","36,241.80",0.00,0.00,0.00,"36,241.80",0.00,0.00,"20,000.00",0.00,1,0.00,6.00,"703,548,288.00",6.00,2014-02-11,"""a55475b1""",2015-09-02,"""CL""","""COL""",2015-09-03,"""P111_135_181""",2016-06-16,2016-06-16,"""P33_146_175""",1994-10-15,"""WIDOWED""",2015-09-30,"""POS""",false,false,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""K"""
"148,808",0.00,"5,112.20","40,000.00",0.00,null,0.00,null,null,null,"80,000.00","19,781.40",0.00,"40,000.00",1.00,2,"19,781.40",36.00,null,36.00,2018-11-10,"""a55475b1""",2016-02-25,null,"""CAL""",2018-11-13,"""P142_57_166""",null,2019-08-13,"""P97_36_170""",2012-09-18,"""MARRIED""",2016-03-25,"""CASH""",false,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"101,005",0.00,"5,847.00",null,null,null,0.00,null,null,null,"150,000.00",null,0.00,"50,000.00",null,1,null,60.00,null,60.00,null,"""P94_109_143""",2022-09-12,null,"""CAL""",null,"""P39_22_51""",null,null,"""a55475b1""",null,null,2022-10-15,"""CASH""",false,null,"""P46_145_78""","""a55475b1""","""a55475b1""","""P94_109_143""","""D"""
"101,065",0.00,"7,798.40",null,null,0.00,"10,660.00",0.00,0.00,0.00,"200,000.00",null,0.00,"30,000.00",null,2,null,48.00,"840,488,064.00",48.00,2022-09-20,"""P19_105_83""",2017-06-27,"""AC""","""REL""",2022-09-20,"""P133_68_68""",null,null,"""a55475b1""",null,null,2022-10-02,"""NDF""",false,false,"""P46_145_78""","""a55475b1""","""P198_131_9""","""P94_109_143""","""T"""
"101,146",0.00,"7,800.80",1.00,0.00,null,0.00,null,null,null,"200,000.00","10,500.46",0.00,"30,000.00",0.00,3,"11,158.20",48.00,null,48.00,2022-06-24,"""a55475b1""",2015-10-23,null,"""CAL""",2022-09-03,"""a55475b1""",null,2022-08-23,"""a55475b1""",2015-07-15,"""SINGLE""",2022-07-24,"""CASH""",false,null,"""P46_145_78""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""


In [19]:
total_past_depth = (
    read_and_cast("applprev_2_","past_depth")
)
total_past_depth

case_id,past_depth_num_group1,past_depth_num_group2,past_depth_cacccardblochreas_147M,past_depth_conts_type_509L,past_depth_credacc_cards_status_52L
u32,u16,u16,cat,str,str
"1,841,684",0,1,"""a55475b1""","""PRIMARY_MOBILE""","""ACTIVE"""
"218,253",0,2,"""a55475b1""","""PHONE""",null
"1,361,272",7,1,"""a55475b1""","""PRIMARY_MOBILE""","""INACTIVE"""
…,…,…,…,…,…
"57,551",1,1,null,"""PRIMARY_MOBILE""",null
"57,552",0,2,null,"""PRIMARY_MOBILE""",null
"57,549",0,2,null,"""PHONE""",null


In [20]:
total_static_base = (
    read_and_cast("static_0","static_base")
)
total_static_base

case_id,static_base_actualdpdtolerance_344P,static_base_amtinstpaidbefduel24m_4187115A,static_base_annuity_780A,static_base_annuitynextmonth_57A,static_base_applicationcnt_361L,static_base_applications30d_658L,static_base_applicationscnt_1086L,static_base_applicationscnt_464L,static_base_applicationscnt_629L,static_base_applicationscnt_867L,static_base_avgdbddpdlast24m_3658932P,static_base_avgdbddpdlast3m_4187120P,static_base_avgdbdtollast24m_4525197P,static_base_avgdpdtolclosure24_3658938P,static_base_avginstallast24m_3658937A,static_base_avglnamtstart24m_4525187A,static_base_avgmaxdpdlast9m_3716943P,static_base_avgoutstandbalancel6m_4187114A,static_base_avgpmtlast12m_4525200A,static_base_clientscnt12m_3712952L,static_base_clientscnt3m_3712950L,static_base_clientscnt6m_3712949L,static_base_clientscnt_100L,static_base_clientscnt_1022L,static_base_clientscnt_1071L,static_base_clientscnt_1130L,static_base_clientscnt_136L,static_base_clientscnt_157L,static_base_clientscnt_257L,static_base_clientscnt_304L,static_base_clientscnt_360L,static_base_clientscnt_493L,static_base_clientscnt_533L,static_base_clientscnt_887L,static_base_clientscnt_946L,static_base_cntincpaycont9m_3716944L,…,static_base_credtype_322L,static_base_datefirstoffer_1144D,static_base_datelastinstal40dpd_247D,static_base_datelastunpaid_3546854D,static_base_disbursementtype_67L,static_base_dtlastpmtallstes_4499206D,static_base_equalitydataagreement_891L,static_base_equalityempfrom_62L,static_base_firstclxcampaign_1125D,static_base_firstdatedue_489D,static_base_inittransactioncode_186L,static_base_isbidproduct_1095L,static_base_isbidproductrequest_292L,static_base_isdebitcard_729L,static_base_lastactivateddate_801D,static_base_lastapplicationdate_877D,static_base_lastapprcommoditycat_1041M,static_base_lastapprcommoditytypec_5251766M,static_base_lastapprdate_640D,static_base_lastcancelreason_561M,static_base_lastdelinqdate_224D,static_base_lastrejectcommoditycat_161M,static_base_lastrejectcommodtypec_5251769M,static_base_lastrejectdate_50D,static_base_lastrejectreason_759M,static_base_lastrejectreasonclient_4145040M,static_base_lastrepayingdate_696D,static_base_lastst_736L,static_base_maxdpdinstldate_3546855D,static_base_opencred_647L,static_base_paytype1st_925L,static_base_paytype_783L,static_base_payvacationpostpone_4187118D,static_base_previouscontdistrict_112M,static_base_twobodfilling_608L,static_base_typesuite_864L,static_base_validfrom_1069D
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,str,date,date,date,str,date,bool,bool,date,date,str,bool,bool,bool,date,date,cat,cat,date,cat,date,cat,cat,date,cat,cat,date,str,date,bool,str,str,date,cat,str,str,date
"613,738",null,null,"2,470.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,1.00,0.00,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,…,"""COL""",null,null,null,"""SBA""",null,null,null,null,null,"""POS""",false,null,null,null,null,"""a55475b1""","""a55475b1""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""a55475b1""","""a55475b1""",null,null,null,null,"""OTHER""","""OTHER""",null,"""a55475b1""","""FO""",null,null
"2,644,298",0.00,"9,567.45","3,045.00",0.00,0.00,0.00,0.00,0.00,0.00,1.00,-2.00,null,-2.00,0.00,"1,467.00",null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,…,"""REL""",2015-10-26,null,2018-09-09,"""DD""",2018-09-10,null,null,2016-01-31,2014-10-27,"""NDF""",true,null,false,2017-12-13,2017-12-10,"""P12_6_178""","""a55475b1""",2017-12-10,"""a55475b1""",2018-09-09,"""a55475b1""","""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""K""",2018-04-09,false,"""OTHER""","""OTHER""",null,"""P19_11_176""","""FO""",null,2019-11-12
"1,388,562",0.00,"20,542.80","1,997.60",0.00,0.00,0.00,0.00,0.00,0.00,1.00,-7.00,null,null,1.00,"2,054.40",null,1.00,0.00,null,0.00,0.00,0.00,0.00,0

In [21]:
total_person_shallow = (
    read_and_cast("person_1","person_shallow")
)
total_person_shallow

case_id,person_shallow_childnum_185L,person_shallow_mainoccupationinc_384A,person_shallow_num_group1,person_shallow_personindex_1023L,person_shallow_persontype_1072L,person_shallow_persontype_792L,person_shallow_birth_259D,person_shallow_birthdate_87D,person_shallow_contaddr_district_15M,person_shallow_contaddr_matchlist_1032L,person_shallow_contaddr_smempladdr_334L,person_shallow_contaddr_zipcode_807M,person_shallow_education_927M,person_shallow_empl_employedfrom_271D,person_shallow_empl_employedtotal_800L,person_shallow_empl_industry_691L,person_shallow_empladdr_district_926M,person_shallow_empladdr_zipcode_114M,person_shallow_familystate_447L,person_shallow_gender_992L,person_shallow_housetype_905L,person_shallow_housingtype_772L,person_shallow_incometype_1044T,person_shallow_isreference_387L,person_shallow_language1_981M,person_shallow_maritalst_703L,person_shallow_registaddr_district_1083M,person_shallow_registaddr_zipcode_184M,person_shallow_relationshiptoclient_415T,person_shallow_relationshiptoclient_642T,person_shallow_remitter_829L,person_shallow_role_1084L,person_shallow_role_993L,person_shallow_safeguarantyflag_411L,person_shallow_sex_738L,person_shallow_type_25L
u32,f32,f32,u16,f32,f32,f32,date,date,cat,bool,bool,cat,cat,date,str,str,cat,cat,str,str,str,str,cat,bool,cat,str,cat,cat,cat,cat,bool,str,str,bool,str,str
"886,374",null,"30,000.00",3,2.00,5.00,5.00,1990-07-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2016-01-17,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""PARENT""","""SPOUSE""",false,"""PE""",null,true,"""F""","""PHONE"""
"898,974",null,"25,000.00",2,1.00,5.00,5.00,1987-12-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-01-15,"""MORE_FIVE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""PRIVATE_SECTOR_EMPLOYEE""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SIBLING""","""SIBLING""",false,"""PE""",null,true,"""M""","""PHONE"""
"1,671,547",null,"34,000.00",0,0.00,1.00,1.00,1979-02-01,null,"""P19_11_176""",false,false,"""P11_15_81""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,"""PRIVATE_SECTOR_EMPLOYEE""",null,"""P10_39_147""",null,"""P19_11_176""","""P11_15_81""",null,null,null,"""CL""",null,true,"""M""","""PRIMARY_MOBILE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"57,543",null,"36,000.00",2,1.00,5.00,5.00,1996-08-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2018-02-15,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""OTHER""","""OTHER""",false,"""PE""",null,false,"""M""","""PHONE"""
"57,551",null,"24,000.00",2,1.00,5.00,5.00,1990-08-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-01-15,"""MORE_FIVE""","""OTHER""","""a55475b1""","""a55475b1""","""MARRIED""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""FRIEND""","""FRIEND""",false,"""CL""",null,false,"""M""","""PHONE"""
"57,549",null,"15,000.00",3,1.00,5.00,5.00,1992-03-01,null,"""a55475b1""",false,false,"""a55475b1""","""a55475b1""",2019-05-04,"""MORE_ONE""","""OTHER""","""a55475b1""","""a55475b1""","""SINGLE""",null,null,null,"""EMPLOYED""",null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SIBLING""","""SIBLING""",false,"""EM""",null,false,"""M""","""PRIMARY_MOBILE"""


In [22]:
total_person_depth = (
    read_and_cast("person_2","person_depth")
)
total_person_depth

case_id,person_depth_num_group1,person_depth_num_group2,person_depth_addres_district_368M,person_depth_addres_role_871L,person_depth_addres_zip_823M,person_depth_conts_role_79M,person_depth_empls_economicalst_849M,person_depth_empls_employedfrom_796D,person_depth_empls_employer_name_740M,person_depth_relatedpersons_role_762T
u32,u16,u16,cat,str,cat,cat,cat,date,cat,cat
"149,963",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"1,718,359",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"928,071",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
…,…,…,…,…,…,…,…,…,…,…
"57,635",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"57,636",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
"57,632",0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null


In [23]:
total_other_shallow = (
    read_and_cast("other_1","other_shallow")
)
total_other_shallow

case_id,other_shallow_amtdebitincoming_4809443A,other_shallow_amtdebitoutgoing_4809440A,other_shallow_amtdepositbalance_4809441A,other_shallow_amtdepositincoming_4809444A,other_shallow_amtdepositoutgoing_4809442A,other_shallow_num_group1
u32,f32,f32,f32,f32,f32,u16
"1,794,503",0.00,0.00,"3,015.20",0.00,24.80,0
"2,685,983","10,000.00","10,000.00",0.00,0.00,0.00,0
"1,918,757",0.00,0.00,0.00,0.00,12.20,0
…,…,…,…,…,…,…
"57,932",0.00,0.00,"23,212.00",0.00,178.20,0
"57,635",0.00,0.00,"12,028.80",0.00,"4,011.00",0
"57,762",0.00,0.00,240.20,0.00,1.80,0


In [24]:
total_deposit_shallow = (
    read_and_cast("deposit_1","deposit_shallow")
)
total_deposit_shallow

case_id,deposit_shallow_amount_416A,deposit_shallow_num_group1,deposit_shallow_contractenddate_991D,deposit_shallow_openingdate_313D
u32,f32,u16,date,date
"656,867","4,239.86",1,2016-12-12,2013-12-13
"1,766,180",278.73,0,2017-09-21,2014-09-22
"2,543,898",264.46,0,2018-02-06,2015-02-07
…,…,…,…,…
"57,890","1,178.49",0,2018-03-11,2015-03-12
"57,774",733.01,0,null,2016-03-03
"57,737",202.00,0,2018-05-16,2016-05-13


In [25]:
total_debitcard_shallow = (
    read_and_cast("debitcard","card_shallow")
)
total_debitcard_shallow

case_id,card_shallow_last180dayaveragebalance_704A,card_shallow_last180dayturnover_1134A,card_shallow_last30dayturnover_651A,card_shallow_num_group1,card_shallow_openingdate_857D
u32,f32,f32,f32,u16,date
"202,926",null,null,null,1,2015-03-09
"1,477,544",null,null,null,1,2014-01-07
"1,522,498",null,null,null,0,2017-04-14
…,…,…,…,…,…
"57,968",null,null,null,0,2017-05-08
"57,932",null,null,null,0,2017-07-06
"57,774",null,null,null,0,2016-03-03


In [21]:
total_credit_internal_shallow = (
    read_and_cast("bureau_a_1_","int_shallow")
)
total_credit_internal_shallow

ComputeError: cannot cast numeric types to 'Categorical'

In [ ]:
total_credit_internal_depth = (
    read_and_cast("bureau_a_2","int_depth")
)
total_credit_internal_depth

case_id,max_collater_valueofguarantee_1124L,max_collater_valueofguarantee_876L,max_num_group1,max_num_group2,max_pmts_dpd_1073P,max_pmts_dpd_303P,max_pmts_overdue_1140A,max_pmts_overdue_1152A,mode_collater_typofvalofguarant_298M,mode_collater_typofvalofguarant_407M,mode_collaterals_typeofguarante_359M,mode_collaterals_typeofguarante_669M,mode_pmts_month_158T,mode_pmts_month_706T,mode_pmts_year_1139T,mode_pmts_year_507T,mode_subjectroles_name_541M,mode_subjectroles_name_838M
u32,f64,f64,u16,u16,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
"1,282,761",0.00,null,3,35,22.00,null,"5,999.40",null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""5.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
"598,914",0.00,null,0,23,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""1.0""",null,"""2018.0""",null,"""a55475b1""","""a55475b1"""
"1,283,407",0.00,null,0,23,0.00,null,0.00,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""10.0""",null,"""2019.0""",null,"""a55475b1""","""a55475b1"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"1,893,033",0.00,0.00,9,35,22.00,24.00,"5,269.88","5,621.60","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""2.0""","""2.0""","""2020.0""","""2019.0""","""a55475b1""","""a55475b1"""
"1,004,831",0.00,"142,000.00",8,35,47.00,108.00,"28,274.51","37,575.57","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""11.0""","""3.0""","""2020.0""","""2013.0""","""a55475b1""","""a55475b1"""
"245,108",0.00,0.00,4,35,1.00,1.00,"1,616.40","4,780.60","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""1.0""","""9.0""","""2020.0""","""2017.0""","""a55475b1""","""a55475b1"""


In [ ]:
total_credit_external_shallow = (
    read_and_cast("bureau_b_1","ext_shallow")
)
total_credit_external_shallow

case_id,max_amount_1115A,max_credlmt_1052A,max_credlmt_228A,max_credlmt_3940954A,max_credquantity_1099L,max_credquantity_984L,max_debtpastduevalue_732A,max_debtvalue_227A,max_dpd_550P,max_dpd_733P,max_dpdmax_851P,max_installmentamount_644A,max_installmentamount_833A,max_instlamount_892A,max_interesteffectiverate_369L,max_interestrateyearly_538L,max_maxdebtpduevalodued_3940955A,max_num_group1,max_numberofinstls_810L,max_overdueamountmax_950A,max_pmtdaysoverdue_1135P,max_pmtnumpending_403L,max_residualamount_1093A,max_residualamount_127A,max_residualamount_3940956A,max_totalamount_503A,max_totalamount_881A,mode_classificationofcontr_1114M,mode_contractdate_551D,mode_contractmaturitydate_151D,mode_contractst_516M,mode_contracttype_653M,mode_credor_3940957M,mode_dpdmaxdatemonth_804T,mode_dpdmaxdateyear_742T,mode_lastupdate_260D,mode_overdueamountmaxdatemonth_494T,mode_overdueamountmaxdateyear_432T,mode_periodicityofpmts_997L,mode_periodicityofpmts_997M,mode_pmtmethod_731M,mode_purposeofcred_722M,mode_subjectrole_326M,mode_subjectrole_43M
u32,f32,f32,f32,f32,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f64,f64,f32,u16,f64,f32,f32,f64,f32,f32,f32,f32,f32,cat,date,date,cat,cat,cat,cat,cat,date,cat,cat,str,cat,cat,cat,cat,cat
"1,905,791","67,367.80",null,null,null,1.00,2.00,0.00,"6,796.84",0.00,0.00,0.00,0.00,"6,796.84","6,966.80",null,42.00,0.00,0,12.00,0.00,0.00,1.00,null,null,null,"67,367.80","64,303.20","""ea6782cc""",2019-09-17,2020-09-17,"""7241344e""","""4257cbed""","""P0_31_66""","""10.0""","""2019.0""",2020-09-02,"""10.0""","""2019.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
"143,809","400,000.00","216,836.00",null,"196,836.00",2.00,1.00,0.00,"414,066.69",0.00,0.00,"6,584.00",0.00,"414,066.69","18,133.33",null,null,0.20,2,22.00,4.00,0.00,22.00,null,"85,830.67","85,830.67","400,000.00","20,447.40","""ea6782cc""",2013-06-14,2025-06-10,"""7241344e""","""1c9c5356""","""b619fa46""","""5.0""","""2018.0""",2019-07-25,"""1.0""","""2019.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""a55475b1"""
"1,654,374","148,844.61",null,null,null,1.00,5.00,0.00,"133,095.67",0.00,0.00,0.00,0.00,"133,095.67","7,324.80",null,42.00,0.00,0,36.00,0.00,0.00,28.00,null,null,null,"148,844.61","146,242.20","""ea6782cc""",2019-03-12,2022-03-12,"""7241344e""","""4257cbed""","""P0_31_66""","""4.0""","""2019.0""",2019-12-11,"""4.0""","""2019.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"241,705",null,"23,010.00",null,"23,010.00",1.00,4.00,0.00,null,0.00,0.00,"51,705.00",0.00,"21,954.16",null,null,null,10.60,0,null,10.60,0.00,null,null,"21,954.16","21,954.16","23,010.00","115,143.80","""ea6782cc""",2019-09-24,2021-09-24,"""7241344e""","""1c9c5356""","""b619fa46""","""3.0""","""2020.0""",2020-08-06,"""3.0""","""2020.0""",null,"""a55475b1""","""a55475b1""","""60c73645""","""ab3c25cf""","""ab3c25cf"""
"163,310","125,600.00",null,0.00,null,3.00,2.00,0.00,"60,964.62",0.00,0.00,"21,476.00",0.00,"160,605.97","5,054.00",32.29,25.90,0.60,2,60.00,0.60,0.00,31.00,0.00,null,null,"257,210.70","77,938.20","""ea6782cc""",2017-10-20,2022-05-01,"""7241344e""","""4257cbed""","""P133_127_114""","""5.0""","""2019.0""",2019-10-08,"""5.0""","""2019.0""",null,"""a0b598e4""","""dbcbe8f8""","""96a8fdfe""","""a55475b1""","""ab3c25cf"""
"126,883","411,720.22","49,168.00","10,000.00","49,168.00",6.00,20.00,"5,691.18","407,298.47",0.00,0.00,"51,275.00",0.00,"40,195.32","10,600.00",0.12,0.12,6.00,6,240.00,8.60,5.00,223.00,0.00,"40,195.32","40,195.32","49,168.00","1,196,136.88","""01f63ac8""",2018-08-20,2019-07-23,"""7241344e""","""4257cbed""","""a7a613e0""","""4.0""","""2018.0""",2019-05-13,"""9.0""","""2018.0""",null,"""a0b598e4""","""f6e26148""","""60c73645""","""a55475b1""","""a55475b1"""


In [ ]:
total_credit_external_depth = (
    read_and_cast("bureau_b_2","ext_depth")
)
total_credit_external_depth

case_id,max_num_group1,max_num_group2,max_pmts_dpdvalue_108P,max_pmts_pmtsoverdue_635A,mode_pmts_date_1107D
u32,u16,u16,f32,f32,date
"1,010,262",5,14,0.00,0.00,2020-08-15
"885,868",1,5,0.00,0.00,2019-08-15
"865,414",0,25,"50,150.00",11.80,2018-08-15
…,…,…,…,…,…
"1,601,655",3,36,"2,949,446.00",21.60,2019-06-15
"1,779,533",2,35,0.00,0.00,2019-09-15
"731,275",0,35,0.00,0.00,2018-10-15


In [ ]:
total_registry_a = (
    read_and_cast("registry_a","reg_a")
)
total_registry_a

case_id,max_amount_4527230A,max_num_group1,mode_name_4527232M,mode_recorddate_4527225D
u32,f32,u16,cat,date
"1,562,239",850.00,5,"""9883ae26""",2019-10-16
"1,641,515","3,715.60",5,"""75c8df1b""",2019-12-04
"1,733,010",850.00,1,"""334350d4""",2020-01-18
…,…,…,…,…
"211,542","2,155.60",7,"""ba8550bf""",2020-03-16
"2,623,171","2,733.80",5,"""d613d705""",2019-10-02
"787,254","3,317.80",8,"""41227108""",2019-09-14


In [ ]:
total_registry_b = (
    read_and_cast("registry_b","reg_b")
)
total_registry_b

case_id,max_amount_4917619A,max_num_group1,mode_deductiondate_4917603D,mode_name_4917606M
u32,f32,u16,date,cat
"993,863","18,400.00",5,2020-06-09,"""4d2a2ad9"""
"245,671","17,058.60",5,2020-05-13,"""abae903b"""
"1,838,552","8,895.40",5,2020-04-24,"""a314a065"""
…,…,…,…,…
"1,942,078","56,199.40",12,2020-04-14,"""d492b9eb"""
"2,683,053","23,248.00",5,2020-06-02,"""74da09fb"""
"1,921,865","109,175.00",5,2020-05-11,"""2983205f"""


In [ ]:
total_registry_c = (
    read_and_cast("registry_c","reg_c")
)
total_registry_c

case_id,max_num_group1,max_pmtamount_36A,mode_employername_160M,mode_processingdate_168D
u32,u16,f32,cat,date
"2,607,582",5,"2,854.27","""32dd4008""",2019-08-07
"686,610",7,"1,809.91","""f37ec779""",2018-12-04
"1,550,297",5,"12,635.20","""e7d51f8d""",2019-10-07
…,…,…,…,…
"750,538",5,"2,905.34","""4eaf6879""",2019-05-22
"705,846",5,"3,815.02","""aa536852""",2019-05-08
"1,292,306",5,"4,023.75","""fc69f41e""",2018-10-18


In [ ]:
# try:
#     for _ in globals().keys():
#         if "total_" in _:
#             print(_)
# except RuntimeError:
#     for _ in globals().keys():
#         if "total_" in _:
#             print(_)

total_base
total_past_shallow
total_past_depth
total_static_base
total_person_shallow
total_person_depth
total_other_shallow
total_deposit_shallow
total_debitcard_shallow
total_credit_internal_shallow
total_credit_internal_depth
total_credit_external_shallow
total_credit_external_depth
total_registry_a
total_registry_b
total_registry_c


In [ ]:
total_df = (
    total_base
    .join(
        total_past_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_past_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_static_base,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_person_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_other_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_deposit_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_debitcard_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_internal_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_shallow,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_credit_external_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_a,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_b,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        total_registry_c,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .with_columns(
        (pl.col(pl.Date) - pl.col("Date")).dt.total_days()
    )
    .drop(["Date"])
    .pipe(select_impuatable)
)
total_df

DuplicateError: unable to hstack, column with name "max_num_group1_right" already exists

In [ ]:
kernel = mf.ImputationKernel(
    total_df.to_pandas(),
    datasets=3,
    save_all_iterations=True,
    random_state=2024
)